## Homework 09:  Parallel Programming 02

## Due Date: Apr 19, 2023, 11:59pm

#### Firstname Lastname: Seonhye Yang

#### E-mail: sy3420@nyu.edu

#### Enter your solutions and submit this notebook

---

**Problem 1 (40p)**

In this problem the goal is to calculate the mean and standard deviation of a large list of numbers by using Reduction as one of Collective Operations, see Lecture 11. 


Consider $N = 256000$ random variables uniform on $[0, 1]$, call these $x_0, x_1, \dots, x_{N - 1}$.  


Write an MPI program with $N=16$ processes that outputs the average and standard deviation of $x_0, x_1, \dots, x_{N - 1}$.


To simplify the problem, let one process, say `Process 0`, independently draws $N$ samples uniformly on $[0, 1]$.

How do you explain the results?


**Instructions:** 
Your program should use MPI4PY and collective operations. 
Save your program as 2020_spring_sol09_pr01.py and run it from the terminal as:

```
mpirun -n 16 python 2020_spring_sol09_pr01.py
```


In [112]:
%%writefile 2023_spring_sol09_pr01.py

from mpi4py import MPI
import numpy as np
import statistics


comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

n = 256000

if rank == 0:
    data = np.random.random(n).astype('f').reshape(size, int(n/size))    
#     data = float(np.random.random(n).reshape(size, int(n/size)))  
    
if rank != 0:
    data = None

zeros = np.zeros(int(n/size), dtype='f')


comm.Scatter(data,zeros, root = 0)

data = comm.bcast(data, root=0)
comm.Bcast(data, root=0)

# n = size * 2
# m = 5
# if rank == 0:
#     A = np.random.randn(n, m)
#     x = np.random.randn(m)

#     ytarget = A.dot(x) # to check the result
    
#     A = A.reshape(size, -1, m)
# else:
#     A = None
#     x = np.zeros(m)

# Asmall = np.zeros((n // size, m))

# # Asmall = comm.scatter(A, root=0)
# comm.Scatter(A, Asmall, root=0)

# # x = comm.bcast(x, root=0)
# comm.Bcast(x, root=0)

# print('rank', rank, ':', Asmall.shape, x.shape)

globalsums = comm.allreduce(sum(zeros), MPI.SUM)
avg = globalsums / n


diff = sum(map(lambda i: (i - avg)**2, zeros))

globaldiff = comm.reduce(diff, MPI.SUM, 0)

print(rank, sum(zeros))
# print('==================================================================')
if rank == 0:
    std = np.sqrt(globaldiff / n)
#     print('==================================================================')
    print('Mean = ', avg, 'Standard deviation =', std)
else:
    None

Overwriting 2023_spring_sol09_pr01.py


In [113]:
!mpirun -n 16 python3 2023_spring_sol09_pr01.py

3 7969.072311292005
11 8032.97866295241
9 7990.850889774534
10 8031.202948563525
12 8022.812618252021
14 8041.512061482206
13 8047.899874919218
7 8004.5025486057275
5 8013.518146395676
2 7942.786519835076
6 8016.321328463593
8 7953.971118859212
15 7978.104012428164
4 7957.217837076101
1 7961.435503704095
0 7982.784157105474
Mean =  0.49979285367073845 Standard deviation 0.2880715477530028


I would say that this has a mean of 0.4999 and standard deviation of 0.288. Standard deviation is pretty low which indicates that the distribution is pretty squished. The results are printed only in the root process. The standard deviations are printed in the root process shows the mean and standard deviations are computed using MPI. 


---
**Problem 2 (60p)**

In this problem the goal is to demonstrate how one can use a Domain Decomposition and  Collective Operations. 

Consider the exponential distribution $X \sim \textrm{Exp}(1)$ with the unit mean. Find numerical approximations of moments of the exponential random variable. 

That is, for a random variable $X$ with the distribution $f(x) = e^{-x}$ for $x \geq 0$, compute the first $15$ moments, where the $k$-th moment is defined as:
$$I_k = \int_{0}^{\infty} x^k f(x) dx.$$


Your program should use MPI parallel collective instructions, where the integration is performed in parallel over $N=16$ processes, over a finite range $[0, M)$, where $M=1000$, with $N = 16$ partitions and $1000$ increments per partition, see Lecture 10 and 11.

Provide evaluations of $J_1, J_2, \dots, J_{15}$, where $$J_k = \int_{0}^{M} x^k f(x) dx.$$


**Instructions:** 

Save your program as 2020_sol09_pr02.py; and run it from the terminal as:

```
mpirun -n 16 python 2020_spring_sol09_pr02.py
```


**Bonus Question (10 points):** 

What is the value of $I_k$, as a function of $k$? How can it be derived?

In [100]:
%%writefile 2020_spring_sol09_pr02.py

import numpy as np
from numpy import pi, exp, sqrt
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# def integral(a_r, h, n):
#     integ = 0.0
#     for j in range(n):
#         t = a_r + (j + 0.5) * h
#         integ += sin(t) * h
#     return integ

def integral(a_r, h, n, K):
    
    integ = np.zeros(K)
    for j in range(n):
        t = a_r + (j + 0.5) * h
        integs = []
        for i in range(1, K+1):
            integs.append(t**i*exp(-t))
        integ = integ+np.array(integs)
    integ = h*integ
    return integ



# a = 0.0
# b = numpy.pi / 2
# dest = 0
# my_int = numpy.zeros(1)
# integral_sum = numpy.zeros(1)
K = 16
M = 1000
integral_sum = np.zeros(16)
n = np.array(1000)
h = M/(n*size) 
a_r = rank*h*n

my_int = integral(a_r, h, n, K)

comm.Reduce(my_int, integral_sum, MPI.SUM, root=0)

# print("Process ", rank, " has the partial integral ", my_int[0])

# Initialize value of n only if this is rank 0
if rank == 0:
    for i in range(K):
        print((i, integral_sum[i]))


Overwriting 2020_spring_sol09_pr02.py


In [101]:
!mpirun -n 16 python 2020_spring_sol09_pr02.py

(0, 1.0001627047952104)
(1, 2.0000001112238226)
(2, 5.99999988885252)
(3, 23.999999999771028)
(4, 120.00000000022848)
(5, 719.9999999999999)
(6, 5040.000000000001)
(7, 40320.00000000004)
(8, 362879.9999999999)
(9, 3628800.0000000023)
(10, 39916799.99999997)
(11, 479001600.0000007)
(12, 6227020800.000004)
(13, 87178291200.0002)
(14, 1307674368000.0007)
(15, 20922789887999.992)
